# Motifs

### Branch and Bound 

In [ ]:
import random

def motif_score(seqs, posicoes, L):
     """
    Calcula o score dos motifs extraídos de seqs através das posições 

    Parâmetros:
        seqs (list of str): Lista de sequências de DNA
        posicoes (list of int): Lista de posições iniciais para extrair os motifs
        L (int): Tamanho do motif

    Return:
        int: O score
    """
     motifs = [seq[p:p+L] for seq, p in zip(seqs, posicoes)]
     return sum(max(col.count(base) for base in set(col)) for col in zip(*motifs))

def random_dna_seq(n):
    """
    Gera uma sequência aleatória de DNA com comprimento n

    Parâmetros:
        n (int): Comprimento da sequência a gerar

    Retorna:
        str: Uma sequência aleatória composta pelas letras A, C, G e T
    """
    return ''.join(random.choice("ACGT") for _ in range(n))

def procura_exaustiva_motifs(seqs, L):
    """
    Procura os melhores motifs de um determinado comprimento nas sequências fornecidas,
    ao testar exaustivamente todas as posições possíveis e selecionar a combinação
    com a maior pontuação.

    Parâmetros:
        seqs (list): Lista das sequências biológicas (strings)
        L (int): Comprimento do motif a procurar

    Retorna:
        Um tuplo com:
            - melhor_p (list): Posições iniciais dos melhores motifs em cada sequência
            - melhor_score (int): Pontuação do melhor conjunto de motifs
    """
    melhor_score, melhor_p = 0, None 
    tam_seq = len(seqs[0]) 
    limite = tam_seq - L + 1  

    def score(pos):
        """
        Calcula a pontuação de conservação para um conjunto de motivos nas posições especificadas
        
        Parâmetros:
           pos (list): Lista de posições iniciais em cada sequência
            
        Retorna:
           int: Pontuação total de conservação das colunas dos motifs
        """
        motifs = [seq[p:p+L] for seq, p in zip(seqs, pos)] 
        return sum(max(col.count(b) for b in set(col)) for col in zip(*motifs))  

    def procura(i, pos_at):
        """
        Função recursiva que realiza a procura por todas as combinações possíveis
        de posições iniciais nas sequências

        Parâmetros:
            i (int): Índice da sequência atual que está a ser processada
            pos_at (list): Lista de posições iniciais já escolhidas para as sequências anteriores
            
        Retorna:
            A função não retorna valores, atualiza as variáveis melhor_score e melhor_p
        """
        nonlocal melhor_score, melhor_p 
        if i == len(seqs): 
            sc = score(pos_at)  
            if sc > melhor_score:  
                melhor_score, melhor_p = sc, pos_at  
            return
        for p in range(limite):  
            procura(i + 1, pos_at + [p])  
    procura(0, [])  
    return melhor_p, melhor_score  

if __name__ == '__main__':
    seqs = "ATGGTCGC TTGTCTGA CCGTAGTA".split()  
    L = 3                                     
    melhor, score_final = procura_exaustiva_motifs(seqs, L)
    print("Posições:", melhor, "Score:", score_final) 

Posições: [3, 2, 2] Score: 8


### Consensus

In [ ]:
def score_motifs(seqs, pos, L):
    """
    Calcula o score dos motifs extraídos de seqs através das posições em pos.
    O score é a soma, para cada coluna do alinhamento dos motifs, da frequência da letra mais comum.

    Parâmetros:
        seqs (list of str): Lista de sequências de DNA.
        pos (list of int): Lista de posições iniciais para extrair os motifs.
        L (int): Tamanho do motif.

    Retorna:
        int: O score dos motifs.
    """
    motifs = [seq[p:p+L] for seq, p in zip(seqs, pos)] 
    score = 0
    for col in zip(*motifs):  
        score += max(col.count(let) for let in set(col))  
    return score

def consensus_heu(seqs, L):
    """
    Algoritmo heurístico que encontra, de forma greedy, as melhores posições iniciais dos motifs:
      - Para as duas primeiras sequências, testa todas as combinações possíveis.
      - Para cada sequência subsequente, escolhe a posição que maximiza o score dos motifs já formados.

    Parâmetros:
        seqs (list of str): Lista de sequências de DNA.
        L (int): Tamanho do motif.

    Retorna:
        Um tuplo com:
            - list of int: Lista das melhores posições iniciais para os motifs.
            - int: O score final dos motifs.
    """
    if len(seqs) < 2:
        raise ValueError("É necessário ter pelo menos duas sequências.")  

    melhor_pos = None  
    melhor_score = -1  
    
    
    for s1 in range(len(seqs[0]) - L + 1):  
        for s2 in range(len(seqs[1]) - L + 1):  
            posicoes = [s1, s2]  
            score_atual = score_motifs(seqs[:2], posicoes, L)  
            if score_atual > melhor_score:  
                melhor_score = score_atual
                melhor_pos = posicoes.copy()
    
    for i in range(2, len(seqs)):  
        m_posicoes = None 
        melhor_score_i = -1  
        for pos in range(len(seqs[i]) - L + 1):  
            posicoes = melhor_pos + [pos] 
            score_atual = score_motifs(seqs[:i+1], posicoes, L)
            if score_atual > melhor_score_i:  
                melhor_score_i = score_atual
                m_posicoes = pos
        melhor_pos.append(m_posicoes)  
        melhor_score = melhor_score_i  
    score_f = score_motifs(seqs, melhor_pos, L)
    return melhor_pos, score_f  

if __name__ == '__main__':
    seqs = ["ATGGTCGC", "TTGTCTGA", "CCGTAGTA"] 
    L = 3  
    pos_f, score_f = consensus_heu(seqs, L)
    print("Posições:", pos_f)  
    print("Score:", score_f)  

Posições: [3, 2, 2]
Score: 8


### Gibbs Sampling

In [ ]:
import random

def score(posicoes, sequencias, L):
    """
    Calcula a pontuação para um conjunto de motifs.

    Parâmetros:
        posicoes (list): Lista de posições iniciais em cada sequência.
        sequencias (list): Lista de sequências de DNA.
        L (int): Comprimento do motif.
        
    Retorna:
        int: Pontuação total dos motifs encontrados.
    """
    motifs = [seq[p:p+L] for seq, p in zip(sequencias, posicoes)]  
    return sum(max(col.count(b) for b in set(col)) for col in zip(*motifs))  

def calcula_probabilidade(seq, L, pi, outros_motifs):
    """
    Calcula a probabilidade do motif a partir da posição inicial na sequência,
    usando um modelo baseado nas contagens de bases das restantes sequências.

    Parâmetros:
        seq (str): A sequência onde se procura o motif.
        L (int): Comprimento do motif.
        pi (int): Posição de início do motif na sequência.
        outros_motifs (list[str]): Lista dos motifs das outras sequências, usada para calcular as frequências das bases.

    Retorna:
        float: A probabilidade do motif começar na posição especificada.
    """
    motif = seq[pi:pi + L]  
    prob = 1.0
    total = len(outros_motifs) + 4  
    for i, base in enumerate(motif):  
        counts = {n: 1 for n in "ACGT"} 
        for m in outros_motifs:  
            counts[m[i].upper()] = counts.get(m[i].upper(), 1) + 1
        prob *= counts[base.upper()] / total  
    return prob

def gibbs_sampling(sequences, L, num_it=1000):
    """
    Realiza a procura de motifs nas sequências utilizando o algoritmo de Gibbs Sampling.

    Parâmetros:
        sequences (list[str]): Lista de sequências onde se pretende encontrar os motifs.
        L (int): Comprimento do motif.
        num_it (int, opcional): Número de iterações do algoritmo.

    Retorna:
        Um tuplo com:
            - list[int]: Lista das melhores posições de início dos motifs em cada sequência.
            - int: A pontuação final dos motifs encontrados.
    """
    
    motif_pos = [random.randint(0, len(seq) - L) for seq in sequences]
    m_score_val = score(motif_pos, sequences, L)  
    m_positions = motif_pos.copy()  

    for _ in range(num_it):  
        for i in range(len(sequences)):  #
            exc_seq = sequences[i]  
            outros_motifs = [
                sequences[j][motif_pos[j]:motif_pos[j] + L] 
                for j in range(len(sequences)) if j != i  
            ]
            probabilidades = [
                calcula_probabilidade(exc_seq, L, pos, outros_motifs)
                for pos in range(len(exc_seq) - L + 1)
            ]
            total_prob = sum(probabilidades)  
            if total_prob == 0:
                probabilidades = [1 / len(probabilidades)] * len(probabilidades)
            else:
                probabilidades = [p / total_prob for p in probabilidades]
            motif_pos[i] = random.choices(range(len(probabilidades)), weights=probabilidades)[0]
        score_atual = score(motif_pos, sequences, L)
        if score_atual > m_score_val:  
            m_score_val = score_atual
            m_positions = motif_pos.copy()
    return m_positions, m_score_val 

if __name__ == "__main__":
    sequences = "ATGGTCGC TTGTCTGA CCGTAGTA".split()
    L = 3
    gibbs_pos, gibbs_m_score = gibbs_sampling(sequences, L, num_it=1000)
    print("Posições:", gibbs_pos, "Score:", gibbs_m_score)

Posições: [3, 2, 5] Score: 8


### Procura exaustiva

In [5]:
def score_motif(seqs, indices, L):
    """
    Calcula o score de uma configuração de motifs

    Parâmetros:
        seqs (list[str]): Lista de sequências de onde os motifs são extraídos
        indices (list[int]): Lista com as posições de início dos motifs em cada sequência
        L (int): Comprimento do motif

    Return:
        int: O score
    """
    
    motifs = [seq[i:i+L] for seq, i in zip(seqs, indices)]
    total_score = 0 
    for col in zip(*motifs):
        total_score += max(col.count(base) for base in set(col))
    return total_score  

def prox_combinacao(indices, seq_comp, L):
    """
    Gera a próxima combinação de índices para os motifs

    Parâmetros:
        indices (list[int]): Combinação atual de índices 
        seq_comp (list[int]): Lista com os comprimentos de cada sequência
        L (int): Comprimento do motif

    Return:
        list[int] ou None: A próxima combinação de índices se existir; caso contrário, None se não houver mais combinações
    """
    novos_indices = indices.copy()  
    pos = len(novos_indices) - 1  
    while pos >= 0 and novos_indices[pos] >= seq_comp[pos] - L:
        pos -= 1 
    if pos < 0:  
        return None 
    novos_indices[pos] += 1 
    for i in range(pos + 1, len(novos_indices)):
        novos_indices[i] = 0
    return novos_indices  

def procura_exaustiva(seqs, L):
    """
    Executa uma procura exaustiva nas sequências para encontrar a configuração de posições que maximiza o score

    Parâmetros:
        seqs (list[str]): Lista de sequências de onde se pretende extrair os motifs
        L (int): Comprimento do motif

    Return:
        Um tuplo com:
            - list[int]: A configuração de índices que maximiza o score
            - int: O score máximo obtido
    """
    seq_lengths = [len(seq) for seq in seqs]  
    indices = [0] * len(seqs)  
    melhor_score = -1  
    melhores_indices = None  
    while indices is not None:
        s = score_motif(seqs, indices, L)  
        if s > melhor_score:  
            melhor_score = s 
            melhores_indices = indices.copy()  
        indices = prox_combinacao(indices, seq_lengths, L)  
    return melhores_indices, melhor_score  

if __name__ == '__main__':
    sequencias = ["ATGGTCGC", "TTGTCTGA", "CCGTAGTA"]  
    L = 3  
    melhores_indices, melhor_score = procura_exaustiva(sequencias, L)  
    print("Melhores posições:", melhores_indices)  
    print("Score:", melhor_score)  

Melhores posições: [3, 2, 2]
Score: 8
